In [1]:
from dataset import get_path, get_subjects, epoch_data
from utils import decod, correlate, match_list
import mne_bids
from pathlib import Path
import pandas as pd
import numpy as np
import mne
import spacy

nlp = spacy.load('fr_core_news_sm')

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import RidgeCV
from wordfreq import zipf_frequency
from Levenshtein import editops
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
mne.set_log_level(False)

In [2]:
report = mne.Report()
path = get_path('LPP_read')
subjects = get_subjects(path)
RUN = 1
subject = subjects[0]
print(f"Subject {subject}'s decoding started")
epochs = []
for run_id in range(1, RUN + 1):
    print(".", end="")
    epo = epoch_data(subject, "%.2i" % run_id, task='listen', path=path)
    epo.metadata["label"] = f"run_{run_id}"
    epochs.append(epo)

# Quick fix for the dev_head: has to be
# fixed before doing source reconstruction
for epo in epochs:
    epo.info["dev_head_t"] = epochs[0].info["dev_head_t"]
    # epo.info['nchan'] = epochs[0].info['nchan']

epochs = mne.concatenate_epochs(epochs)

# Get the evoked potential averaged on all epochs for each channel
evo = epochs.average(method="median")
evo.plot(spatial_colors=True)

# Handling the data structure
epochs.metadata["kind"] = epochs.metadata.trial_type.apply(
    lambda s: eval(s)["kind"]
)
epochs.metadata["word"] = epochs.metadata.trial_type.apply(
    lambda s: eval(s)["word"]
)
epochs.metadata['closing'] = epochs.metadata.closing_.fillna(0)
# Run a linear regression between MEG signals
# and word frequency classification
X = epochs.get_data()


y = epochs.metadata['closing']

R_vec = decod(X, y)
R_vec_avg = np.mean(R_vec,axis = 1)

fig, ax = plt.subplots(1, figsize=[6, 6])
dec = plt.fill_between(epochs.times, R_vec_avg)
# plt.show()
report.add_evokeds(evo, titles=f"Evoked for sub {subject} ")
report.add_figure(fig, title=f"decoding for subject {subject}")
# report.add_figure(dec, subject, tags="word")
report.save("./figures/reading_decoding_closing.html", open_browser=False, overwrite=True)

print("Finished!")



Subject 1's decoding started
.Running the script on RAW data:
run 01, subject: 1


/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Omitted 4 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw = mne_bids.rea

      index  Unnamed: 0     word   onset  duration  \
0         0           0  Lorsque    0.70       0.3   
1         1           1  j'avais    1.05       0.3   
2         2           2      six    1.40       0.3   
3         3           3     ans,    1.75       0.3   
4         4           4     j'ai    2.10       0.3   
...     ...         ...      ...     ...       ...   
1406   1459        1459       ne  511.35       0.3   
1407   1460        1460     peut  511.70       0.3   
1408   1461        1461      pas  512.05       0.3   
1409   1462        1462    aller  512.40       0.3   
1410   1463        1463     bien  512.75       0.3   

                               trial_type    start condition word_index  \
0     {'kind': 'word', 'word': 'Lorsque'}   44.534  sentence       None   
1      {'kind': 'word', 'word': 'javais'}   44.889  sentence       None   
2         {'kind': 'word', 'word': 'six'}   45.189  sentence          3   
3         {'kind': 'word', 'word': 'ans'}   45.489 

/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:112: RuntimeWarning: The measurement information indicates a low-pass frequency of 20.0 Hz. The decim=20 parameter will result in a sampling frequency of 50.0 Hz, which can cause aliasing artifacts.
  epochs = mne.Epochs(raw, **mne_events(meta, raw), decim=20, tmin=-0.2, tmax=0.8)


RuntimeError: By default, MNE does not load data into main memory to conserve resources. adding, dropping, or reordering channels requires epochs data to be loaded. Use preload=True (or string) in the constructor or epochs.load_data().

In [2]:
report = mne.Report()
path = get_path('LPP_read')
subjects = get_subjects(path)
RUN = 1

for subject in subjects:

    print(f"Subject {subject}'s decoding started")
    epochs = []
    for run_id in range(1, RUN + 1):
        print(".", end="")
        epo = epoch_data(subject, "%.2i" % run_id, task='listen', path=path)
        epo.metadata["label"] = f"run_{run_id}"
        epochs.append(epo)

    # Quick fix for the dev_head: has to be
    # fixed before doing source reconstruction
    for epo in epochs:
        epo.info["dev_head_t"] = epochs[0].info["dev_head_t"]
        # epo.info['nchan'] = epochs[0].info['nchan']

    epochs = mne.concatenate_epochs(epochs)

    # Get the evoked potential averaged on all epochs for each channel
    evo = epochs.average(method="median")
    evo.plot(spatial_colors=True)

    # Handling the data structure
    epochs.metadata["kind"] = epochs.metadata.trial_type.apply(
        lambda s: eval(s)["kind"]
    )
    epochs.metadata["word"] = epochs.metadata.trial_type.apply(
        lambda s: eval(s)["word"]
    )
    epochs.metadata['closing'] = epochs.metadata.closing_.fillna(0)
    # Run a linear regression between MEG signals
    # and word frequency classification
    X = epochs.get_data()


    y = epochs.metadata['closing']

    R_vec = decod(X, y)
    R_vec_avg = np.mean(R_vec,axis = 1)

    fig, ax = plt.subplots(1, figsize=[6, 6])
    dec = plt.fill_between(epochs.times, R_vec_avg)
    # plt.show()
    report.add_evokeds(evo, titles=f"Evoked for sub {subject} ")
    report.add_figure(fig, title=f"decoding for subject {subject}")
    # report.add_figure(dec, subject, tags="word")
    report.save("./figures/reading_decoding_embeddings.html", open_browser=False, overwrite=True)

    print("Finished!")
    



Subjects for which the decoding will be tested: 

['1', '2', '3', '4', '5', '6']
Subject 1's decoding started
.Running the script on RAW data:
run 01, subject: 1


/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Omitted 4 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/media/cb271805/272a9f9d-f140-48d6-b390-094ccc29aae0/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw = mne_bids.rea

<Epochs |  1411 events (good & bad), -0.2 - 0.8 sec, baseline -0.2 – 0 sec, ~5.3 MB, data not loaded, with metadata,
 '1': 1411>


KeyError: 'Event name "kind=="word"" could not be found. The following events are present in the data: 1 The epochs.metadata Pandas query did not yield any results: cannot insert level_0, already exists'